# Human in the Loop

Ability to replace messages. It looks for msg with the same id to replace them.

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from uuid import uuid4


memory = SqliteSaver.from_conn_string(":memory:")

In [14]:
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    ## assign ids to message that dont have them
    
    for message in right: 
        if not message.id:
            message.id = str(uuid4())
    ## merge the new messages with the existing messages
    merged = left.copy()
    for message in right: 
        for idx, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any exisitng messages with the same id
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]
    


In [8]:
tool = TavilySearchResults(max_results=2)

/var/folders/8l/9f8bkrp10_j_nv1019x4c4lr0000gn/T/ipykernel_58500/4289725543.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


In [16]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm",    self.call_openai, defer=True)
        graph.add_node("action", self.take_action, defer=True)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]  ## this is passed before the action node (where tools are called)
                                         ## this is where the human interrupts any tools that might not want to be called
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [17]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [22]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)

    messages = [HumanMessage(content="Whats the weather in SF?")]
    thread = {"configurable": {"thread_id": "1"}}
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    
    print(abot.graph.get_state(thread))
    print(abot.graph.get_state(thread).next)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='dbad97a8-923b-4e78-b1bb-6de10fe8972d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qFA1P8EFaiQpiRgQAWoJvqBH', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bqn1FhD3JAHQtpvkHQwfO4b5VvFwN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6fe9d5ee-2b00-4580-a971-a5f97bd883dc-0', tool_calls=[{'name': 'tavily_search_results_json', '

In [ ]:
# the following code below allows us to allow the Human to interrupt before calling a tool.

In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
abot.graph.get_state(thread).next

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)
        

# State Memory

- a snapshot it stored for each state: `AgentState`, `thread`, `thread_ts`

<img src="thread_snapshot.png" alt="Thread Snapshot" title="Thread Snapshot" width="400">

- `g.get_state_history(thread)` rturn siterator over all `StateSnapshots`
- `g.invoke(None, {...,thread,thread_ts,..})`
- `g.stream(None, {...,thread,thread_ts,..})`
- `g.get_state({thread, thread_ts})` 
    - to modify it: `g.update_state(thread, state1m.values)` if you run stream it will use the modified state as the starting point

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)
current_values = abot.graph.get_state(thread)
current_values.values['messages'][-1]
current_values.values['messages'][-1].tool_calls

In [ ]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [ ]:
abot.graph.update_state(thread, current_values.values)
abot.graph.get_state(thread)
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

In [ ]:
## Time Travel
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

To fetch the same state as was filmed, the offset below is changed to `-3` from `-1`. This accounts for the initial state `__start__` and the first state that are now stored to state memory with the latest version of software.

In [ ]:
to_replay = states[-3]

In [ ]:
to_replay

In [ ]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

# Go back in time and edit

In [ ]:
to_replay

In [ ]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [ ]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

In [ ]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

# Add Message to a state at a given time


In [ ]:
to_replay

In [ ]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [ ]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [ ]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [ ]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)